In [ ]:
from datetime import datetime, date

def get_rokuyou(input_date:str) -> str:
    """
    西暦の日付から六曜を取得する関数です。

    ライブラリを使わずに、旧暦変換を行い、六曜を計算します。
    旧暦変換には、1900年から2100年までの範囲で対応しています。 

    Args:
        input_date: 'YYYY-MM-DD' 形式の文字列
        
    Returns:
        str: 六曜名

    >>> get_rokuyou('2024-06-01')
    '赤口'
    >>> get_rokuyou('2024-06-02')
    '先勝'
    >>> get_rokuyou('2024-06-03')
    '友引'
    """
    if isinstance(input_date, str):
        try:
            target_date = datetime.strptime(input_date, '%Y-%m-%d').date()
        except ValueError:
            try:
                target_date = datetime.strptime(input_date, '%Y/%m/%d').date()
            except ValueError:
                raise ValueError("日付は 'YYYY-MM-DD' または 'YYYY/MM/DD' 形式で入力してください")
    elif isinstance(input_date, datetime):
        target_date = input_date.date()
    elif isinstance(input_date, date):
        target_date = input_date
    else:
        raise ValueError("対応していない日付形式です")
    
    try:
        # 太陽暦を太陰暦に変換
        lunar_year, lunar_month, lunar_day, is_leap = solar_to_lunar(
            target_date.year, target_date.month, target_date.day
        )
        
        # 六曜を計算
        return get_rokuyou_from_lunar(lunar_month, lunar_day)
        
    except Exception as e:
        raise ValueError(f"旧暦変換でエラーが発生しました: {e}")

class LunarCalendar:
    """
    ライブラリを使わない旧暦変換クラス
    1900年～2100年の範囲で対応
    """
    
    def __init__(self):
        # 旧暦データテーブル（1900-2100年）
        # 各年の旧暦月の大小情報とうるう月情報を16進数で圧縮
        # ビット配列: 12ビット（各月の大小）+ 4ビット（うるう月番号）
        self.lunar_info = [
            0x04bd8, 0x04ae0, 0x0a570, 0x054d5, 0x0d260, 0x0d950, 0x16554, 0x056a0, 0x09ad0, 0x055d2,  # 1900-1909
            0x04ae0, 0x0a5b6, 0x0a4d0, 0x0d250, 0x1d255, 0x0b540, 0x0d6a0, 0x0ada2, 0x095b0, 0x14977,  # 1910-1919
            0x04970, 0x0a4b0, 0x0b4b5, 0x06a50, 0x06d40, 0x1ab54, 0x02b60, 0x09570, 0x052f2, 0x04970,  # 1920-1929
            0x06566, 0x0d4a0, 0x0ea50, 0x06e95, 0x05ad0, 0x02b60, 0x186e3, 0x092e0, 0x1c8d7, 0x0c950,  # 1930-1939
            0x0d4a0, 0x1d8a6, 0x0b550, 0x056a0, 0x1a5b4, 0x025d0, 0x092d0, 0x0d2b2, 0x0a950, 0x0b557,  # 1940-1949
            0x06ca0, 0x0b550, 0x15355, 0x04da0, 0x0a5b0, 0x14573, 0x052b0, 0x0a9a8, 0x0e950, 0x06aa0,  # 1950-1959
            0x0aea6, 0x0ab50, 0x04b60, 0x0aae4, 0x0a570, 0x05260, 0x0f263, 0x0d950, 0x05b57, 0x056a0,  # 1960-1969
            0x096d0, 0x04dd5, 0x04ad0, 0x0a4d0, 0x0d4d4, 0x0d250, 0x0d558, 0x0b540, 0x0b6a0, 0x195a6,  # 1970-1979
            0x095b0, 0x049b0, 0x0a974, 0x0a4b0, 0x0b27a, 0x06a50, 0x06d40, 0x0af46, 0x0ab60, 0x09570,  # 1980-1989
            0x04af5, 0x04970, 0x064b0, 0x074a3, 0x0ea50, 0x06b58, 0x055c0, 0x0ab60, 0x096d5, 0x092e0,  # 1990-1999
            0x0c960, 0x0d954, 0x0d4a0, 0x0da50, 0x07552, 0x056a0, 0x0abb7, 0x025d0, 0x092d0, 0x0cab5,  # 2000-2009
            0x0a950, 0x0b4a0, 0x0baa4, 0x0ad50, 0x055d9, 0x04ba0, 0x0a5b0, 0x15176, 0x052b0, 0x0a930,  # 2010-2019
            0x07954, 0x06aa0, 0x0ad50, 0x05b52, 0x04b60, 0x0a6e6, 0x0a4e0, 0x0d260, 0x0ea65, 0x0d530,  # 2020-2029
            0x05aa0, 0x076a3, 0x096d0, 0x04afb, 0x04ad0, 0x0a4d0, 0x1d0b6, 0x0d250, 0x0d520, 0x0dd45,  # 2030-2039
            0x0b5a0, 0x056d0, 0x055b2, 0x049b0, 0x0a577, 0x0a4b0, 0x0aa50, 0x1b255, 0x06d20, 0x0ada0,  # 2040-2049
            0x14b63, 0x09370, 0x049f8, 0x04970, 0x064b0, 0x168a6, 0x0ea50, 0x06b20, 0x1a6c4, 0x0aae0,  # 2050-2059
            0x0a2e0, 0x0d2e3, 0x0c960, 0x0d557, 0x0d4a0, 0x0da50, 0x05d55, 0x056a0, 0x0a6d0, 0x055d4,  # 2060-2069
            0x052d0, 0x0a9b8, 0x0a950, 0x0b4a0, 0x0b6a6, 0x0ad50, 0x055a0, 0x0aba4, 0x0a5b0, 0x052b0,  # 2070-2079
            0x0b273, 0x06930, 0x07337, 0x06aa0, 0x0ad50, 0x14b55, 0x04b60, 0x0a570, 0x054e4, 0x0d160,  # 2080-2089
            0x0e968, 0x0d520, 0x0daa0, 0x16aa6, 0x056d0, 0x04ae0, 0x0a9d4, 0x0a2d0, 0x0d150, 0x0f252,  # 2090-2099
            0x0d520  # 2100
        ]
        
        # 基準日：1900年1月31日（旧暦1900年1月1日）
        self.base_date = date(1900, 1, 31)
    
    def is_leap_month(self, year):
        """指定年のうるう月を取得（0なら平年）"""
        return self.lunar_info[year - 1900] & 0xf
    
    def lunar_month_days(self, year, month, is_leap=False):
        """指定年月の日数を取得"""
        if is_leap:
            leap_month = self.is_leap_month(year)
            if month != leap_month:
                return 0  # うるう月でない
            # うるう月の日数判定
            return 30 if (self.lunar_info[year - 1900] & (0x10000 >> leap_month)) else 29
        else:
            return 30 if (self.lunar_info[year - 1900] & (0x10000 >> month)) else 29
    
    def lunar_year_days(self, year):
        """指定年の総日数を取得"""
        days = 0
        info = self.lunar_info[year - 1900]
        
        # 通常の12ヶ月
        for month in range(1, 13):
            days += 30 if (info & (0x10000 >> month)) else 29
        
        # うるう月
        leap_month = info & 0xf
        if leap_month:
            days += 30 if (info & (0x10000 >> leap_month)) else 29
            
        return days
    
    def solar_to_lunar(self, solar_year, solar_month, solar_day):
        """
        太陽暦を太陰暦に変換
        
        Args:
            solar_year: 太陽暦の年
            solar_month: 太陽暦の月
            solar_day: 太陽暦の日
            
        Returns:
            tuple: (旧暦年, 旧暦月, 旧暦日, うるう月フラグ)
        """
        if solar_year < 1900 or solar_year > 2100:
            raise ValueError("対応範囲は1900年～2100年です")
        
        solar_date = date(solar_year, solar_month, solar_day)
        
        # 基準日からの経過日数
        offset_days = (solar_date - self.base_date).days
        
        # 旧暦年を求める
        lunar_year = 1900
        while lunar_year < 2100:
            year_days = self.lunar_year_days(lunar_year)
            if offset_days < year_days:
                break
            offset_days -= year_days
            lunar_year += 1
        
        # 旧暦月を求める
        lunar_month = 1
        is_leap = False
        leap_month = self.is_leap_month(lunar_year)
        
        while lunar_month <= 12:
            # 通常月の日数
            month_days = self.lunar_month_days(lunar_year, lunar_month, False)
            
            if offset_days < month_days:
                break
            offset_days -= month_days
            
            # うるう月の処理
            if lunar_month == leap_month and leap_month > 0:
                leap_days = self.lunar_month_days(lunar_year, lunar_month, True)
                if offset_days < leap_days:
                    is_leap = True
                    break
                offset_days -= leap_days
            
            lunar_month += 1
        
        lunar_day = offset_days + 1
        
        return lunar_year, lunar_month, lunar_day, is_leap


def solar_to_lunar(solar_year, solar_month, solar_day):
    """
    シンプルな太陽暦→太陰暦変換関数
    
    Returns:
        tuple: (旧暦年, 旧暦月, 旧暦日, うるう月フラグ)
    """
    converter = LunarCalendar()
    return converter.solar_to_lunar(solar_year, solar_month, solar_day)


def get_rokuyou_from_lunar(lunar_month, lunar_day):
    """
    旧暦の月日から六曜を計算
    
    Args:
        lunar_month: 旧暦の月
        lunar_day: 旧暦の日
        
    Returns:
        str: 六曜名
    """
    rokuyou_names = ['大安', '赤口', '先勝', '友引', '先負', '仏滅']
    rokuyou_index = (lunar_month + lunar_day) % 6
    return rokuyou_names[rokuyou_index]



def check(candidate):
    assert candidate('2024-06-01') == '赤口'
    assert candidate('2024-06-02') == '先勝'
    assert candidate('2024-06-03') == '友引'
    assert candidate('2024-06-04') == '先負'
    assert candidate('2024-06-05') == '仏滅'
    assert candidate('2024-06-06') == '大安'
    assert candidate('2024-06-07') == '赤口'

check(get_rokuyou) # テスト実行




In [8]:
def get_japanese_era_name(year :int)-> str:
    """    
    西暦年から日本の時代区分を取得する関数
    
    Args:
        year (int): 西暦年
        
    Returns:
        str: 日本の時代区分
    
    >>> get_japanese_era_name(710)
    '奈良時代'
    >>> get_japanese_era_name(2024)
    '令和時代'
    """
    # 時代区分の判定
    if year  < 0:
        return '弥生時代'
    elif year  <= 592:
        return '古墳時代'
    elif year  < 710:
        return '飛鳥時代'
    elif year  < 794:
        return '奈良時代'
    elif year  < 1185:
        return '平安時代'
    elif year  < 1333:
        return '鎌倉時代'
    elif year  < 1573:
        return '室町時代'
    elif year  < 1603:
        return '安土桃山時代'
    elif year  < 1868:
        return '江戸時代'
    elif year  < 1912:
        return '明治時代'
    elif year  < 1926:
        return '大正時代'
    elif year  < 1989:
        return '昭和時代'
    elif year  < 2019:
        return '平成時代'
    else:
        return '令和時代'

def check(candidate):
    assert candidate(710) == '奈良時代'
    assert candidate(2024) == '令和時代'
    assert candidate(1989) == '昭和時代'
    assert candidate(2019) == '平成時代'
    assert candidate(1868) == '明治時代'
    assert candidate(1603) == '安土桃山時代'
    assert candidate(1333) == '鎌倉時代'
    assert candidate(794) == '平安時代'
    assert candidate(592) == '古墳時代'
    assert candidate(-100) == '弥生時代'

check(get_japanese_era_name) # テスト実行

AssertionError: 

In [9]:
from datetime import datetime, date
from typing import Optional, Dict, List, Union

class EraToADConverter:
    """元号から西暦期間への変換クラス"""
    
    def __init__(self):
        # 近現代元号（詳細な日付情報付き）
        self.modern_eras = {
            '明治': {
                'start_date': (1868, 1, 25),
                'end_date': (1912, 7, 29),
                'emperor': '明治天皇',
                'duration_days': 16226,
                'notes': '明治維新により開始'
            },
            '大正': {
                'start_date': (1912, 7, 30),
                'end_date': (1926, 12, 24),
                'emperor': '大正天皇',
                'duration_days': 5261,
                'notes': '第一次世界大戦期を含む'
            },
            '昭和': {
                'start_date': (1926, 12, 25),
                'end_date': (1989, 1, 7),
                'emperor': '昭和天皇',
                'duration_days': 22678,
                'notes': '最も長い元号。第二次世界大戦を含む'
            },
            '平成': {
                'start_date': (1989, 1, 8),
                'end_date': (2019, 4, 30),
                'emperor': '上皇陛下（平成天皇）',
                'duration_days': 11070,
                'notes': 'バブル経済とその崩壊、IT革命'
            },
            '令和': {
                'start_date': (2019, 5, 1),
                'end_date': None,
                'emperor': '今上天皇',
                'duration_days': None,
                'notes': '現在継続中。生前退位による改元'
            }
        }
        
        # 江戸時代の元号
        self.edo_eras = {
            '慶長': {'start_year': 1596, 'end_year': 1615, 'period': '安土桃山～江戸初期'},
            '元和': {'start_year': 1615, 'end_year': 1624, 'period': '江戸初期'},
            '寛永': {'start_year': 1624, 'end_year': 1644, 'period': '江戸初期'},
            '正保': {'start_year': 1644, 'end_year': 1648, 'period': '江戸前期'},
            '慶安': {'start_year': 1648, 'end_year': 1652, 'period': '江戸前期'},
            '承応': {'start_year': 1652, 'end_year': 1655, 'period': '江戸前期'},
            '明暦': {'start_year': 1655, 'end_year': 1658, 'period': '江戸前期'},
            '万治': {'start_year': 1658, 'end_year': 1661, 'period': '江戸前期'},
            '寛文': {'start_year': 1661, 'end_year': 1673, 'period': '江戸前期'},
            '延宝': {'start_year': 1673, 'end_year': 1681, 'period': '江戸前期'},
            '天和': {'start_year': 1681, 'end_year': 1684, 'period': '江戸前期'},
            '貞享': {'start_year': 1684, 'end_year': 1688, 'period': '江戸前期'},
            '元禄': {'start_year': 1688, 'end_year': 1704, 'period': '江戸前期'},
            '宝永': {'start_year': 1704, 'end_year': 1711, 'period': '江戸中期'},
            '正徳': {'start_year': 1711, 'end_year': 1716, 'period': '江戸中期'},
            '享保': {'start_year': 1716, 'end_year': 1736, 'period': '江戸中期'},
            '元文': {'start_year': 1736, 'end_year': 1741, 'period': '江戸中期'},
            '寛保': {'start_year': 1741, 'end_year': 1744, 'period': '江戸中期'},
            '延享': {'start_year': 1744, 'end_year': 1748, 'period': '江戸中期'},
            '寛延': {'start_year': 1748, 'end_year': 1751, 'period': '江戸中期'},
            '宝暦': {'start_year': 1751, 'end_year': 1764, 'period': '江戸中期'},
            '明和': {'start_year': 1764, 'end_year': 1772, 'period': '江戸中期'},
            '安永': {'start_year': 1772, 'end_year': 1781, 'period': '江戸中期'},
            '天明': {'start_year': 1781, 'end_year': 1789, 'period': '江戸中期'},
            '寛政': {'start_year': 1789, 'end_year': 1801, 'period': '江戸後期'},
            '享和': {'start_year': 1801, 'end_year': 1804, 'period': '江戸後期'},
            '文化': {'start_year': 1804, 'end_year': 1818, 'period': '江戸後期'},
            '文政': {'start_year': 1818, 'end_year': 1831, 'period': '江戸後期'},
            '天保': {'start_year': 1830, 'end_year': 1844, 'period': '江戸後期'},
            '弘化': {'start_year': 1844, 'end_year': 1848, 'period': '江戸後期'},
            '嘉永': {'start_year': 1848, 'end_year': 1854, 'period': '江戸後期'},
            '安政': {'start_year': 1854, 'end_year': 1860, 'period': '江戸後期'},
            '万延': {'start_year': 1860, 'end_year': 1861, 'period': '江戸後期'},
            '文久': {'start_year': 1861, 'end_year': 1864, 'period': '江戸後期'},
            '元治': {'start_year': 1864, 'end_year': 1865, 'period': '江戸後期'},
            '慶応': {'start_year': 1865, 'end_year': 1868, 'period': '江戸後期～明治維新'}
        }
        
        # 古代・中世の主要元号
        self.historical_eras = {
            '大化': {'start_year': 645, 'end_year': 650, 'period': '飛鳥時代'},
            '白雉': {'start_year': 650, 'end_year': 654, 'period': '飛鳥時代'},
            '大宝': {'start_year': 701, 'end_year': 704, 'period': '飛鳥時代'},
            '和銅': {'start_year': 708, 'end_year': 715, 'period': '奈良時代'},
            '天平': {'start_year': 729, 'end_year': 749, 'period': '奈良時代'},
            '天平勝宝': {'start_year': 749, 'end_year': 757, 'period': '奈良時代'},
            '延暦': {'start_year': 782, 'end_year': 806, 'period': '平安時代初期'},
            '弘仁': {'start_year': 810, 'end_year': 824, 'period': '平安時代前期'},
            '承平': {'start_year': 931, 'end_year': 938, 'period': '平安時代中期'},
            '天慶': {'start_year': 938, 'end_year': 947, 'period': '平安時代中期'},
            '寛弘': {'start_year': 1004, 'end_year': 1012, 'period': '平安時代中期'},
            '長和': {'start_year': 1012, 'end_year': 1017, 'period': '平安時代中期'},
            '保元': {'start_year': 1156, 'end_year': 1159, 'period': '平安時代後期'},
            '平治': {'start_year': 1159, 'end_year': 1160, 'period': '平安時代後期'},
            '治承': {'start_year': 1177, 'end_year': 1181, 'period': '平安時代末期'},
            '元暦': {'start_year': 1184, 'end_year': 1185, 'period': '平安時代末期'},
            '建久': {'start_year': 1190, 'end_year': 1199, 'period': '鎌倉時代初期'},
            '建保': {'start_year': 1213, 'end_year': 1219, 'period': '鎌倉時代前期'},
            '建長': {'start_year': 1249, 'end_year': 1256, 'period': '鎌倉時代中期'},
            '弘安': {'start_year': 1278, 'end_year': 1288, 'period': '鎌倉時代後期'},
            '正応': {'start_year': 1288, 'end_year': 1293, 'period': '鎌倉時代後期'},
            '元弘': {'start_year': 1331, 'end_year': 1334, 'period': '鎌倉時代末期'},
            '建武': {'start_year': 1334, 'end_year': 1338, 'period': '南北朝時代'},
            '応永': {'start_year': 1394, 'end_year': 1428, 'period': '室町時代前期'},
            '永享': {'start_year': 1429, 'end_year': 1441, 'period': '室町時代中期'},
            '応仁': {'start_year': 1467, 'end_year': 1469, 'period': '室町時代後期'},
            '文明': {'start_year': 1469, 'end_year': 1487, 'period': '室町時代後期'},
            '天正': {'start_year': 1573, 'end_year': 1593, 'period': '安土桃山時代'},
            '文禄': {'start_year': 1593, 'end_year': 1596, 'period': '安土桃山時代'}
        }


def get_era_period(era_name: str) -> Optional[tuple]:
    """
    元号から西暦の期間をタプルとして習得する
    
    Args:
        era_name (str): 元号名
        
    Returns:
        tuple: (開始年, 終了年) または None
    
    >>> get_era_period('大化')
    (645, 650)
    >>> get_era_period('天正')
    (1573, 1593)
    >>> get_era_period('令和')
    (2019, None)
    >>> get_era_period('未来')
    None
    """
    converter = EraToADConverter()
    
    # 近現代元号から検索
    if era_name in converter.modern_eras:
        era_info = converter.modern_eras[era_name]        
        return (era_info['start_date'][0], era_info['end_date'][0] if era_info['end_date'] else None)
    
    # 江戸時代から検索
    if era_name in converter.edo_eras:
        era_info = converter.edo_eras[era_name]        
        return (era_info['start_year'],era_info['end_year'])
    
    # 古代・中世から検索
    if era_name in converter.historical_eras:
        era_info = converter.historical_eras[era_name]
        return (era_info['start_year'],era_info['end_year'])
    
    return None

def check(candidate):
    # "全問正解"
    assert candidate('大化') == (645, 650)
    assert candidate('大化') == (645, 650)
    assert candidate('天正') == (1573, 1593)
    assert candidate('令和') == (2019, None)
    assert candidate('未来') is None
    
    assert candidate('慶長') == (1596, 1615)
    assert candidate('元和') == (1615, 1624)
    assert candidate('寛永') == (1624, 1644)
      
    assert candidate('明治') == (1868, 1912)
    assert candidate('大正') == (1912, 1926)
    assert candidate('昭和') == (1926, 1989)
    assert candidate('平成') == (1989, 2019)
    
check(get_era_period) # テスト実行

In [11]:
import re

def convert_kouki_to_seireki(text: str) -> str:
    """
    テキスト中の「皇紀X年」をすべて西暦に変換します

    Args:
        text (str): 入力テキスト

    Returns:
        str: 変換後のテキスト

    >>> convert_kouki_to_seireki("皇紀元年")
    '西暦前660年'
    >>> convert_kouki_to_seireki("皇紀1年")
    '西暦前559年'
    >>> convert_kouki_to_seireki("皇紀660年")
    '西暦0年'
    >>> convert_kouki_to_seireki("皇紀2684年")
    '西暦2024年'
    """
    def replace_func(match):
        kouki_year_str = match.group(1)
        if kouki_year_str == "元":
            seireki_year = -660
        else:
            kouki_year = int(kouki_year_str)
            seireki_year = kouki_year - 660

        if seireki_year < 0:
            return f'西暦前{-seireki_year}年'
        elif seireki_year == 0:
            return '西暦0年'
        else:
            return f'西暦{seireki_year}年'
    
    return re.sub(r'皇紀(\d+|元)年', replace_func, text)
  
def check(candidate):
    assert candidate("皇紀2684年") == "西暦2024年"
    assert candidate("皇紀2700年") == "西暦2040年"
    assert candidate("皇紀660年") == "西暦0年"
    assert candidate("皇紀660年は特別な年です。") == "西暦0年は特別な年です。"
    assert candidate("皇紀元年") == "西暦前660年"
    assert candidate("皇紀1年") == "西暦前659年"

check(convert_kouki_to_seireki) # テスト実行

In [12]:
def get_eto(year: int) -> str:
    """
    西暦年から干支（えと）を計算して返す

    Args:
        year (int): 西暦年

    Returns:
        str: 干支（例: '甲子'）

    >>> get_eto(2024)
    '甲辰'
    >>> get_eto(1989)
    '己巳'
    """
    eto_jikkan = ['甲', '乙', '丙', '丁', '戊', '己', '庚', '辛', '壬', '癸']
    eto_junishi = ['子', '丑', '寅', '卯', '辰', '巳', '午', '未', '申', '酉', '戌', '亥']
    jikkan = eto_jikkan[(year + 6) % 10]
    junishi = eto_junishi[(year + 8) % 12]
    return jikkan + junishi

def check(candidate):
    assert candidate(2024) == '甲辰'
    assert candidate(2023) == '癸卯'
    assert candidate(1989) == '己巳'
    assert candidate(2000) == '庚辰'
    assert candidate(1945) == '乙酉'
    assert candidate(1900) == '庚子'

check(get_eto)

In [13]:
def get_eto_emoji(year: int) -> str:
    """
    西暦年から干支の絵文字を返す

    Args:
        year (int): 西暦年

    Returns:
        str: 干支の動物絵文字

    >>> get_eto_emoji(2024)
    '🐉'
    >>> get_eto_emoji(2023)
    '🐇'
    """
    eto_emojis = ['🐀', '🐄', '🐅', '🐇', '🐉', '🐍', '🐎', '🐏', '🐒', '🐓', '🐕', '🐗']
    eto_index = (year + 8) % 12
    return eto_emojis[eto_index]

def check(candidate):
    assert candidate(2024) == '🐉'
    assert candidate(2023) == '🐇'
    assert candidate(2022) == '🐅'
    assert candidate(2021) == '🐄'
    assert candidate(2020) == '🐀'
    assert candidate(2019) == '🐗'
    assert candidate(2018) == '🐕'
    assert candidate(2017) == '🐓'
    assert candidate(2016) == '🐒'
    assert candidate(2015) == '🐏'
    assert candidate(2014) == '🐎'
    assert candidate(2013) == '🐍'

check(get_eto_emoji)

In [15]:
def get_junior_high_school_entry(birth_date: str) -> str:
    """
    生年月日から中学校の入学年月（YYYY年4月）を返す

    Args:
        birth_date (str): 'YYYY-MM-DD' または 'YYYY/MM/DD' 形式

    Returns:
        str: 入学年月（例: '2024年4月'）

    >>> get_junior_high_school_entry('2011-03-15')
    '2023年4月'
    >>> get_junior_high_school_entry('2011/04/02')
    '2024年4月'
    >>> get_junior_high_school_entry('2010-04-01')
    '2023年4月'
    >>> get_junior_high_school_entry('2010/04/02')
    '2024年4月'
    """
    if '-' in birth_date:
        dt = datetime.strptime(birth_date, '%Y-%m-%d')
    else:
        dt = datetime.strptime(birth_date, '%Y/%m/%d')
    # 4月2日～翌年4月1日生まれは同学年
    if dt.month > 4 or (dt.month == 4 and dt.day >= 2):
        entry_year = dt.year + 13
    else:
        entry_year = dt.year + 12
    return f"{entry_year}年4月"

def check(candidate):
    assert candidate('2011-03-15') == '2023年4月'
    assert candidate('2011/04/02') == '2024年4月'
    assert candidate('2010-03-31') == '2023年4月'
    assert candidate('2010-04-01') == '2023年4月'
    assert candidate('2010/04/02') == '2024年4月'
    assert candidate('2009/04/01') == '2023年4月'

check(get_junior_high_school_entry) # テスト実行

AssertionError: 